Each report is a list of numbers called levels that are separated by spaces.

In the example above, the reports can be found safe or unsafe by checking those rules:

- `7 6 4 2 1`: Safe because the levels are all decreasing by 1 or 2.
- `1 2 7 8 9`: Unsafe because 2 7 is an increase of 5.
- `9 7 6 2 1`: Unsafe because 6 2 is a decrease of 4.
- `1 3 2 4 5`: Unsafe because 1 3 is increasing but 3 2 is decreasing.
- `8 6 4 4 1`: Unsafe because 4 4 is neither an increase or a decrease.
- `1 3 6 7 9`: Safe because the levels are all increasing by 1, 2, or 3.
So, in this example, 2 reports are safe.

Analyze the unusual data from the engineers. How many reports are safe?

In [120]:
import numpy as np

def is_unsafe(diffs):
    increase_5 = np.any(diffs == 5)
    decrease_4 = np.any(diffs == -4)
    consecutive_same = np.any(diffs == 0)
    increase_then_decrease = np.any((diffs[:-1] > 0) & (diffs[1:] < 0))
    
    return increase_5 or decrease_4 or consecutive_same or increase_then_decrease
    

def is_safe(diffs):
    # I'm annoyed because above they don't specify decrease by 3 is safe
    # but this is the only way to get the accepted answer
    all_decreasing_1_2_or_3 = np.all((diffs == -1) | (diffs == -2) | (diffs == -3))
    all_increasing_1_2_or_3 = np.all((diffs == 1) | (diffs == 2) | (diffs == 3))

    return all_decreasing_1_or_2 or all_increasing_1_2_or_3


def assess(arr):
    # diffs between consecutive elements
    diffs = np.diff(arr)

    if is_unsafe(diffs):
        return False

    if is_safe(diffs):
        return True

    # The rules don't cover all cases ¯\_(ツ)_/¯
    return None


Check against the examples given

In [105]:
a = (7, 6, 4, 2, 1)
b = (1, 2, 7, 8, 9)
c = (9, 7, 6, 2, 1)
d = (1, 3, 2, 4, 5)
e = (8, 6, 4, 4, 1)
f = (1, 3, 6, 7, 9)

In [106]:
assess(a)

True

In [107]:
assess(b)

False

In [108]:
assess(c)

False

In [109]:
assess(d)

False

In [110]:
assess(e)

False

In [111]:
assess(f)

True

In [112]:
from aocd import get_data

I've overcomplicated things here but it was because I was trying to figure out why I had the wrong answer before I allowed decrementing by 3 in the "safe" category.

In [114]:
def count_true(arr):
    arr = np.asarray(arr)
    return np.sum(arr == True)

In [115]:
results = []
for line in get_data(day=2, year=2024).splitlines():
    array = np.fromstring(line, dtype=int, sep=" ")
    safe = assess(array)

    results.append(safe)

In [117]:
count_true(results)

np.int64(639)